In [9]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    x = np.random.uniform(low=-3., high=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + beta * x[:,1] + e
    return y,x,nobs

yn,xn,nobs = gen_data()

In [11]:
def compute_loglike(resid):
    sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
    ll = np.log(stats.norm.pdf(resid,loc=0,scale=sigma))
    return ll

In [12]:
def compute_score(yn,xn,params):
    
    xn = sm.add_constant(xn)
    resid = yn - np.matmul(xn,params)
    
    k = len(params)
    scale = (resid**2).mean()
    
    tile_resid = np.tile( resid, k)
    tile_resid = np.reshape(tile_resid, (k,xn.shape[0]) ).transpose()
    grad = tile_resid*xn/scale     
    return grad
    
def compute_hess(yn,xn,params):
    pass


def setup_shi(yn,xn):
    x1n,x2n = xn[:,0:2],xn[:,1:3]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    k1 = len(model1_fit.params)
    
    ll1 = compute_loglike(model1_fit.resid)
    grad1 =  compute_score(yn,x1n,model1_fit.params)
    hess1 = model1.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    k2 = len(model1_fit.params)
    
    ll2 = compute_loglike(model2_fit.resid)
    grad2 =  compute_score(yn,x2n,model2_fit.params)
    hess2 = model2.hessian(model2_fit.params)
    
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

In [21]:
reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200,use_boot2=False)
print("reg: %s, boot: %s, boot_nml: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2, llr, std, omega ))

reg: [1. 0. 0.], boot: [1. 0. 0.], boot_nml: [1. 0. 0.], llr:-0.11405977231818155, std: 0.7581146091446301, omega:0.039418208092958394


In [15]:
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

[1. 0. 0.]


In [16]:
sys.path.append("../")
import vuong_tests

#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 1*x[:,2] + 1*x[:,0] + beta * x[:,1] + e
    return y,x,nobs

reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.93 0.02 0.05], boot1: [0.93 0.02 0.05], boot2: [0.93 0.02 0.05], llr:-7.977745931875669, std: 32.37427852548365, omega:0.9924989642713996
[0.96 0.04 0.  ]


In [17]:
#generate the sample
def gen_data(beta=1):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 2*x[:,2] + 2*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.946 0.025 0.029], boot1: [0.939 0.023 0.038], boot2: [0.939 0.023 0.038], llr:-0.8578926453547956, std: 31.278492869631307, omega:0.9979682083126672
[0.96 0.01 0.03]


In [22]:
#generate the sample
def gen_data(beta=1):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [1. 0. 0.], boot2: [0.999 0.001 0.   ], llr:-0.04135650681726836, std: 0.9872527748331357, omega:0.03868795907877991
[1. 0. 0.]


In [23]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .1*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:42.68433726681501, std: 8.529128834554964, omega:0.28937214550097784
[0. 1. 0.]


In [24]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .01*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [0.99 0.   0.01], boot2: [0.99 0.   0.01], llr:0.2045795309368879, std: 1.1905161521409517, omega:0.04613293577023034
[1. 0. 0.]


In [25]:
#generate the sample
def gen_data(beta=2):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .01*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [0.99 0.01 0.  ], boot2: [0.99 0.01 0.  ], llr:0.45328590614936987, std: 1.2447358700446813, omega:0.04724574367677426
[1. 0. 0.]
